# Monitoring an ongoing MIBI run

This notebook can be run alongside an active MIBIScope run.  As images are generated, this notebook will automatically pass the data through pre-specified functions, like tiff 
extraction, qc metric computation, etc. Eventually, all of the processing steps in the toffy repo will be incorporated into the Watcher functionality here. For now, this notebook will automatically extract tifs and generate QC plots of your data. 

## This notebook is an example: create a copy before running it or you will get merge conflicts!

In [ ]:
import os
import pandas as pd

from toffy.fov_watcher import start_watcher
from toffy.watcher_callbacks import build_extract_callback, build_qc_callback

## Required variables

 - `run_name` should contain the exact name of the MIBI run that will be monitored. 
 - `panel_path` should point to a panel csv specifying the targets on your panel. For an example, see the [example panel file](https://github.com/angelolab/toffy/blob/main/files/example_panel_file.csv)

In [ ]:
# the name of the run
run_name = 'YYYY-MM-DD_run_name'

# path to user panel
panel_path = 'C:\\Users\\Customer.ION\\Documents\\my_cool_panel.csv'


# these are set automatically
base_dir = os.path.join('D:\\Data', run_name)

# path to log folder
log_path = os.path.join('C:\\Users\\Customer.ION\\Documents\\watcher_logs', run_name)

# load panel
panel = pd.read_csv(panel_path)

## Callback initialization

Callbacks listed in `per_fov` will be run on each completed FoV, the moment both the .bin & .json
are found. Callbacks listed in `per_run` will be run only once all expected FoV's have been
discovered and processed.  Currently, no callbacks have been specified `per_run`.

Each callback will need a location to save its relevant data. This notebooks will generate
that location

`build_extract_callback` specifies that every FoV generate tiffs according to the supplied `panel`.
Additional arguments are `intensities` which when set to True, will extract the intensity data, and
`time_res` which will overwrite the default time resolution.

`build_qc_callback` will run all currently available qc metrics on the FoV.  Additional arguments
are `gaussian_blur` and `blur_factor`.

In [ ]:
extraction_dir = os.path.join('D:\\Extracted_Images', run_name)
qc_dir = os.path.join('C:\\Users\\Customer.ION\\Documents\\run_metrics', run_name)

per_fov = [
    build_extract_callback(extraction_dir, panel, intensities=True),
    build_qc_callback(qc_dir, panel)
]
per_run = []

In [ ]:
start_watcher(base_dir, log_path, per_fov, per_run)